In [1]:
test = ['Register A: 729',
        'Register B: 0',
        'Register C: 0',
        '',
        'Program: 0,1,5,4,3,0']

In [2]:
class computer():
    def __init__(self, data=None):
        self.program = None
        self.pointer = 0
        self.register = {'A':None, 'B':None, 'C':None}
        self.output = ''
        if data is not None:
            self.set_computer(data)
        
    def set_computer(self, data):
        for i in range(0, 3):
            reg, v = data[i].split(': ')
            self.set_register(reg[-1], int(v))
            
        program = data[-1][len('Program: '):]
        self.set_program(program)
        self.output = ''
        
    def reset_computer(self):
        self.program = None
        self.pointer = 0
        self.register = {'A':None, 'B':None, 'C':None}
        self.pointer = 0
        self.output = ''
        
    def set_program(self, program):
        program = program.split(',')
        int_program = []
        for p in program:
            int_program.append(int(p))
        self.program = int_program
        
    def get_register(self, reg):
        return self.register[reg]
    
    def set_register(self, reg, value):
        self.register[reg] = value
        
    def print_register(self, reg=None):
        if reg is None:
            print('Registers :', self.register)
        else:
            print('Register', reg, ':', self.register[reg])
            
    def get_combo(self, operand):
        if operand < 4:
            return operand
        elif operand == 4:
            return self.get_register('A')
        elif operand == 5:
            return self.get_register('B')
        elif operand == 6:
            return self.get_register('C')
        raise Exception("7 is invalid operand")
        
    def dv(self, operand):
        operand = self.get_combo(operand)
        numerator = self.get_register('A')
        denominator = 2**(operand)
        result = int(numerator//denominator)
        return result
        
    def adv(self, operand):
        result = self.dv(operand)
        self.set_register('A', result)
        
    def bxl(self, operand):
        B = self.get_register('B')
        result = B ^ operand
        self.set_register('B', result)
        
    def bst(self, operand):
        operand = self.get_combo(operand)
        result = operand%8
        self.set_register('B', result)
        
    def jnz(self, operand):
        if self.get_register('A') != 0:
            self.pointer = operand-2
        
    def bxc(self, operand):
        B = self.get_register('B')
        C = self.get_register('C')
        result = B ^ C
        self.set_register('B', result)
        
    def out(self, operand):
        operand = self.get_combo(operand)
        value = operand%8
        #print(operand, value)
        if len(self.output) != 0:
            self.output += ','
        self.output += str(value)
        
    def bdv(self, operand):
        result = self.dv(operand)
        self.set_register('B', result)
        
    def cdv(self, operand):
        result = self.dv(operand)
        self.set_register('C', result)
        
    def opcode(self, code, operand):        
        if code == 0: #adv
            self.adv(operand)
            
        elif code == 1: #bxl
            self.bxl(operand)
            
        elif code == 2: #bst
            self.bst(operand)
            
        elif code == 3: #jnz
            self.jnz(operand)
            
        elif code == 4: #bxc
            self.bxc(operand)
            
        elif code == 5: #out
            self.out(operand)
            
        elif code == 6: #bdv
            self.bdv(operand)
            
        elif code == 7: #cdv
            self.cdv(operand)
            
    def run(self, prnt=True):
        while self.pointer < len(self.program):
            code = self.program[self.pointer]
            operand = self.program[self.pointer+1]
            
            self.opcode(code, operand)
            self.pointer += 2
            
        if prnt and len(self.output):
            print('Part 1 result:', self.output)
            
    def run_quine(self):
        og_register = {key: value if not isinstance(value, dict) else {
                        k: v for k, v in value.items()} for key, value in self.register.items()}
        og_program = self.program
        
        A = 0
        pointer = 1
        run = True
        while run:
            self.reset_computer()
            self.register = {key: value if not isinstance(value, dict) else {
                              k: v for k, v in value.items()} for key, value in og_register.items()}
            self.set_register('A', A)
            self.program = og_program
            
            self.run(False)
            output_prog = self.output.split(',')
            if len(output_prog) == 0 or len(output_prog) < pointer:
                A += 1
                continue
                
            int_program = []
            for p in output_prog:
                int_program.append(int(p))
            
            okay = True
            for p in range(0, pointer):
                if int_program[-(p+1)] != og_program[-(p+1)]:
                    okay = False
            
            if okay:
                if pointer == len(og_program):
                    print('Part 2 result:', A)
                    run = False
                else:
                    A *= 8
                    pointer += 1
            else:
                A += 1
                
            if len(int_program) > len(og_program):
                print('FAILED!')
                break

In [3]:
test_comp = computer()
test_comp.set_register('C', 9)
test_comp.set_program('2,6')
test_comp.run()
test_comp.print_register('B')

test_comp.reset_computer()
test_comp = computer()
test_comp.set_register('A', 10)
test_comp.set_program('5,0,5,1,5,4')
test_comp.run()

test_comp = computer()
test_comp.set_register('A', 2024)
test_comp.set_program('0,1,5,4,3,0')
test_comp.run()
test_comp.print_register('A')

test_comp = computer()
test_comp.set_register('B', 29)
test_comp.set_program('1,7')
test_comp.run()
test_comp.print_register('B')

test_comp = computer()
test_comp.set_register('B', 2024)
test_comp.set_register('C', 43690)
test_comp.set_program('4,0')
test_comp.run()
test_comp.print_register('B')

Register B : 1
Part 1 result: 0,1,2
Part 1 result: 4,2,5,6,7,7,7,7,3,1,0
Register A : 0
Register B : 26
Register B : 44354


In [4]:
test_comp = computer(test)
test_comp.run()

Part 1 result: 4,6,3,5,6,3,5,2,1,0


In [5]:
with open('input_day17.txt', 'r') as f:
    data = f.readlines()
    f.close()
    
comp = computer(data)
comp.run()
comp.set_computer(data)
comp.run_quine()

Part 1 result: 1,7,6,5,1,0,5,0,7
Part 2 result: 236555995274861
